**AI Model Security**

Check package details

In [1]:
%%bash
uv pip show model-security-client

Name: model-security-client
Version: 0.3.1
Location: /home/rteixeira/Documents/AI-MODEL-SECURITY/.venv/lib/python3.12/site-packages
Requires: airs-schemas, click, httpx, httpx-retries, modelscan-pai, pydantic, pyjwt
Required-by:


This Script tests authentication + authorization reachability to the AIRS/AIMS control plane endpoints, and if successful it returns the private PyPI repo URL you should use to install the Model Security client packages.

In [2]:
%%bash
./ai-model-security.sh

Step 1: Checking required environment variables...
  ✓ All required environment variables are set
Step 2: Setting API endpoints...
  ✓ Token endpoint: https://auth.apps.paloaltonetworks.com/oauth2/access_token
  ✓ API endpoint: https://api.sase.paloaltonetworks.com/aims
Step 3: Requesting SCM access token...
  → Calling: POST https://auth.apps.paloaltonetworks.com/oauth2/access_token
  → Client ID: rteixeira-airs-model-sec-sa@12312312312.iam.panserviceaccount.com
  → TSG ID: 12312312312
  → HTTP Status: 200
  ✓ SCM access token received
Step 4: Extracting access token from response...
  ✓ Access token extracted successfully
Step 5: Retrieving PyPI repository URL...
  → Calling: GET https://api.sase.paloaltonetworks.com/aims/mgmt/v1/pypi/authenticate
  → HTTP Status: 200
  ✓ PyPI response received successfully
Step 6: Extracting PyPI URL from response...
  ✓ PyPI URL extracted successfully


supressed@us-west1-python.pkg.dev/pairs-modelsec-prd-l9mu/ms-prod-usw1-mp-main-pypi/simple/


Verify AI model Security packages installed in Python venv

In [3]:
%%bash
set -e
.venv/bin/python3 -m pip list | egrep -i "model|security|aims|pai|scan" || true

model-security-client    0.3.1
modelscan                0.8.7
modelscan-pai            1.7.0


## AI Model Security Processing Flow  

<img src="pictures/ai-model-01.png" width="400">

### Scan using CLI

In [4]:
%%bash
sudo -E .venv/bin/model-security scan \
    --security-group-uuid "3ef2ef70-814f-4252-8971-a4985400cc72" \
    --model-path "/home/rteixeira/Documents/AI-MODEL-SECURITY/ollama-gpt-oss-120b" \
    -l source=ollama -l model=gpt-oss_120b -l format=gguf

INFO:model_security_client.credentials:New model-security-client credential context initialized
INFO:model_security_client.api:Scan initiated
INFO:model_security_client.api:Model size calculated
/home/rteixeira/Documents/AI-MODEL-SECURITY/.venv/lib/python3.12/site-packages/numpy/_core/_methods.py:190: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/rteixeira/Documents/AI-MODEL-SECURITY/.venv/lib/python3.12/site-packages/numpy/_core/_methods.py:134: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/home/rteixeira/Documents/AI-MODEL-SECURITY/.venv/lib/python3.12/site-packages/numpy/_core/_methods.py:170: RuntimeWarning: invalid value encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)
INFO:model_security_client.api:Submitting scan for model
INFO:model_security_client.credentials:Requesting access token from token endpoint
INFO:model_security_cli

{
    "uuid": "939d6c50-5998-4cbd-aa11-04f1fd924f2a",
    "tsg_id": "12312312312",
    "created_at": "2026-02-28T13:35:14.859149Z",
    "updated_at": "2026-02-28T13:35:15.002645Z",
    "model_uri": "/home/rteixeira/Documents/AI-MODEL-SECURITY/ollama-gpt-oss-120b",
    "owner": "12312312312",
    "scan_origin": "MODEL_SECURITY_SDK",
    "security_group_uuid": "3ef2ef70-814f-4252-8971-a4985400cc72",
    "security_group_name": "rteixeira-local",
    "model_version_uuid": "17fd62ff-b407-4221-8913-7d3b6b142927",
    "scanner_version": "1.7.0",
    "eval_outcome": "ALLOWED",
    "time_started": "2026-02-28T13:34:19.037550Z",
    "eval_summary": {
        "rules_passed": 7,
        "rules_failed": 0,
        "total_rules": 7
    },
    "error_code": null,
    "error_message": null,
    "model_formats": [
        "gguf"
    ],
    "enabled_rule_count_snapshot": 7,
    "total_files_scanned": 2,
    "total_files_skipped": 0,
    "labels": [
        {
            "key": "model",
            "valu

### GET Scan using CLI


In [5]:
%%bash
sudo -E .venv/bin/model-security get-scan "77042dc4-8921-4c57-8873-3757acfcad55"

INFO:model_security_client.credentials:New model-security-client credential context initialized
INFO:model_security_client.credentials:Requesting access token from token endpoint


{
    "uuid": "77042dc4-8921-4c57-8873-3757acfcad55",
    "tsg_id": "12312312312",
    "created_at": "2026-02-23T14:15:36.188178Z",
    "updated_at": "2026-02-23T14:15:36.390619Z",
    "model_uri": "https://huggingface.co/moonshotai/Kimi-K2.5",
    "owner": "12312312312",
    "scan_origin": "HUGGING_FACE",
    "security_group_uuid": "ed8a56bf-2aed-4af9-a489-5081214f3f67",
    "security_group_name": "rteixeira-HF",
    "model_version_uuid": "6d7d1ae2-78b4-4d74-8421-60838cf78c9f",
    "scanner_version": "1.7.0",
    "eval_outcome": "BLOCKED",
    "time_started": "2026-02-23T14:15:36.183736Z",
    "eval_summary": {
        "rules_passed": 8,
        "rules_failed": 3,
        "total_rules": 11
    },
    "error_code": null,
    "error_message": null,
    "model_formats": [
        "safetensors",
        "not_model",
        "openvino_bin",
        "json",
        "yaml",
        "safetensors_index"
    ],
    "enabled_rule_count_snapshot": 11,
    "total_files_scanned": 72,
    "total_fil

In [6]:
from model_security_client.api import ModelSecurityAPIClient

# Initialize the client
client = ModelSecurityAPIClient(
    base_url="https://api.sase.paloaltonetworks.com/aims"
)


In [7]:
result = client.scan(
    security_group_uuid="ed8a56bf-2aed-4af9-a489-5081214f3f67",
    model_uri="https://huggingface.co/microsoft/DialoGPT-medium",
    labels={ "env": "production" }
)

print(f"Scan completed: {result.eval_outcome}")

Scan completed: EvalOutcome.BLOCKED


In [8]:
from model_security_client.api import ModelSecurityAPIClient

client = ModelSecurityAPIClient(base_url="https://api.sase.paloaltonetworks.com/aims")

result = client.scan(
    security_group_uuid="a961a6a2-6a41-49c6-a6ba-f2142f3ef720",
    model_uri="https://huggingface.co/microsoft/DialoGPT-medium",
    labels={"env": "production"},
)

scan = client.get_scan(result.uuid)

def pretty_scan(scan_obj):
    # Helpers (works with pydantic-like objects)
    def g(name, default=None):
        return getattr(scan_obj, name, default)

    def fmt_dt(dt):
        return dt.isoformat() if dt else "—"

    labels = g("labels", []) or []
    label_str = ", ".join([f"{getattr(l,'key','?')}={getattr(l,'value','?')}" for l in labels]) or "—"

    formats = sorted(set(g("model_formats", []) or []))
    formats_str = ", ".join(formats) if formats else "—"

    summary = g("eval_summary", None)
    passed = getattr(summary, "rules_passed", "—") if summary else "—"
    failed = getattr(summary, "rules_failed", "—") if summary else "—"
    total  = getattr(summary, "total_rules", "—") if summary else "—"

    outcome = str(g("eval_outcome", "—"))
    scan_uuid = g("uuid", "—")
    sg_uuid = g("security_group_uuid", "—")
    sg_name = g("security_group_name", "—")
    model_uri = g("model_uri", "—")
    origin = str(g("scan_origin", "—"))
    source = str(g("source_type", "—"))

    scanned = g("total_files_scanned", "—")
    skipped = g("total_files_skipped", "—")

    created = g("created_at", None)
    updated = g("updated_at", None)
    started = g("time_started", None)

    # Best-effort "why" hint based on what we know (pickle)
    hint = None
    if "BLOCKED" in outcome.upper() and any("pickle" in f for f in formats):
        hint = "Likely blocked by policy: model contains pickle-based PyTorch artifacts (e.g., pytorch_model.bin), which many orgs disallow."

    print("AI Model Security Scan")
    print("────────────────────────────────────────")
    print(f"Outcome:           {outcome}")
    print(f"Scan UUID:         {scan_uuid}")
    print(f"Security Group:    {sg_name} ({sg_uuid})")
    print(f"Model URI:         {model_uri}")
    print(f"Origin / Source:   {origin} / {source}")
    print(f"Rules:             passed={passed}, failed={failed}, total={total}")
    print(f"Files:             scanned={scanned}, skipped={skipped}")
    print(f"Detected formats:  {formats_str}")
    print(f"Labels:            {label_str}")
    print(f"Started:           {fmt_dt(started)}")
    print(f"Created:           {fmt_dt(created)}")
    print(f"Updated:           {fmt_dt(updated)}")
    if hint:
        print("────────────────────────────────────────")
        print(f"Hint:              {hint}")

pretty_scan(scan)

AI Model Security Scan
────────────────────────────────────────
Outcome:           EvalOutcome.BLOCKED
Scan UUID:         7659a329-9ac3-4caf-b236-81f62df89453
Security Group:    Default HUGGING_FACE (a961a6a2-6a41-49c6-a6ba-f2142f3ef720)
Model URI:         https://huggingface.co/microsoft/DialoGPT-medium
Origin / Source:   ScanOrigin.HUGGING_FACE / SourceType.HUGGING_FACE
Rules:             passed=10, failed=1, total=11
Files:             scanned=7, skipped=5
Detected formats:  json, keras_weights, not_model, pickle, pytorch_torch_script, pytorch_v0_1_10, zip
Labels:            env=production
Started:           2026-02-28T13:35:31.256851+00:00
Created:           2026-02-28T13:35:31.260773+00:00
Updated:           2026-02-28T13:35:31.386091+00:00
────────────────────────────────────────
Hint:              Likely blocked by policy: model contains pickle-based PyTorch artifacts (e.g., pytorch_model.bin), which many orgs disallow.


In [9]:
from model_security_client.api import ModelSecurityAPIClient

client = ModelSecurityAPIClient(base_url="https://api.sase.paloaltonetworks.com/aims")

result = client.scan(
    security_group_uuid="a961a6a2-6a41-49c6-a6ba-f2142f3ef720",
    model_uri="https://huggingface.co/nvidia/NVIDIA-Nemotron-3-Nano-30B-A3B-FP8",
    labels={"env": "production"},
)

scan = client.get_scan(result.uuid)

def pretty_scan(scan_obj):
    # Helpers (works with pydantic-like objects)
    def g(name, default=None):
        return getattr(scan_obj, name, default)

    def fmt_dt(dt):
        return dt.isoformat() if dt else "—"

    labels = g("labels", []) or []
    label_str = ", ".join([f"{getattr(l,'key','?')}={getattr(l,'value','?')}" for l in labels]) or "—"

    formats = sorted(set(g("model_formats", []) or []))
    formats_str = ", ".join(formats) if formats else "—"

    summary = g("eval_summary", None)
    passed = getattr(summary, "rules_passed", "—") if summary else "—"
    failed = getattr(summary, "rules_failed", "—") if summary else "—"
    total  = getattr(summary, "total_rules", "—") if summary else "—"

    outcome = str(g("eval_outcome", "—"))
    scan_uuid = g("uuid", "—")
    sg_uuid = g("security_group_uuid", "—")
    sg_name = g("security_group_name", "—")
    model_uri = g("model_uri", "—")
    origin = str(g("scan_origin", "—"))
    source = str(g("source_type", "—"))

    scanned = g("total_files_scanned", "—")
    skipped = g("total_files_skipped", "—")

    created = g("created_at", None)
    updated = g("updated_at", None)
    started = g("time_started", None)

    # Best-effort "why" hint based on what we know (pickle)
    hint = None
    if "BLOCKED" in outcome.upper() and any("pickle" in f for f in formats):
        hint = "Likely blocked by policy: model contains pickle-based PyTorch artifacts (e.g., pytorch_model.bin), which many orgs disallow."

    print("AI Model Security Scan")
    print("────────────────────────────────────────")
    print(f"Outcome:           {outcome}")
    print(f"Scan UUID:         {scan_uuid}")
    print(f"Security Group:    {sg_name} ({sg_uuid})")
    print(f"Model URI:         {model_uri}")
    print(f"Origin / Source:   {origin} / {source}")
    print(f"Rules:             passed={passed}, failed={failed}, total={total}")
    print(f"Files:             scanned={scanned}, skipped={skipped}")
    print(f"Detected formats:  {formats_str}")
    print(f"Labels:            {label_str}")
    print(f"Started:           {fmt_dt(started)}")
    print(f"Created:           {fmt_dt(created)}")
    print(f"Updated:           {fmt_dt(updated)}")
    if hint:
        print("────────────────────────────────────────")
        print(f"Hint:              {hint}")

pretty_scan(scan)

AI Model Security Scan
────────────────────────────────────────
Outcome:           EvalOutcome.BLOCKED
Scan UUID:         c6dc90e3-5ebd-4fc6-af4b-c6f25a69a3c2
Security Group:    Default HUGGING_FACE (a961a6a2-6a41-49c6-a6ba-f2142f3ef720)
Model URI:         https://huggingface.co/nvidia/NVIDIA-Nemotron-3-Nano-30B-A3B-FP8
Origin / Source:   ScanOrigin.HUGGING_FACE / SourceType.HUGGING_FACE
Rules:             passed=10, failed=1, total=11
Files:             scanned=32, skipped=6
Detected formats:  json, not_model, safetensors, safetensors_index, yaml
Labels:            env=production
Started:           2026-02-28T13:35:40.915360+00:00
Created:           2026-02-28T13:35:40.920934+00:00
Updated:           2026-02-28T13:35:41.067507+00:00


In [10]:
from model_security_client.api import ModelSecurityAPIClient

client = ModelSecurityAPIClient(base_url="https://api.sase.paloaltonetworks.com/aims")

result = client.scan(
    security_group_uuid="ed8a56bf-2aed-4af9-a489-5081214f3f67",
    model_uri="https://huggingface.co/nvidia/NVIDIA-Nemotron-3-Nano-30B-A3B-FP8",
    labels={"env": "production"},
)

scan = client.get_scan(result.uuid)

def pretty_scan(scan_obj):
    # Helpers (works with pydantic-like objects)
    def g(name, default=None):
        return getattr(scan_obj, name, default)

    def fmt_dt(dt):
        return dt.isoformat() if dt else "—"

    labels = g("labels", []) or []
    label_str = ", ".join([f"{getattr(l,'key','?')}={getattr(l,'value','?')}" for l in labels]) or "—"

    formats = sorted(set(g("model_formats", []) or []))
    formats_str = ", ".join(formats) if formats else "—"

    summary = g("eval_summary", None)
    passed = getattr(summary, "rules_passed", "—") if summary else "—"
    failed = getattr(summary, "rules_failed", "—") if summary else "—"
    total  = getattr(summary, "total_rules", "—") if summary else "—"

    outcome = str(g("eval_outcome", "—"))
    scan_uuid = g("uuid", "—")
    sg_uuid = g("security_group_uuid", "—")
    sg_name = g("security_group_name", "—")
    model_uri = g("model_uri", "—")
    origin = str(g("scan_origin", "—"))
    source = str(g("source_type", "—"))

    scanned = g("total_files_scanned", "—")
    skipped = g("total_files_skipped", "—")

    created = g("created_at", None)
    updated = g("updated_at", None)
    started = g("time_started", None)

    # Best-effort "why" hint based on what we know (pickle)
    hint = None
    if "BLOCKED" in outcome.upper() and any("pickle" in f for f in formats):
        hint = "Likely blocked by policy: model contains pickle-based PyTorch artifacts (e.g., pytorch_model.bin), which many orgs disallow."

    print("AI Model Security Scan")
    print("────────────────────────────────────────")
    print(f"Outcome:           {outcome}")
    print(f"Scan UUID:         {scan_uuid}")
    print(f"Security Group:    {sg_name} ({sg_uuid})")
    print(f"Model URI:         {model_uri}")
    print(f"Origin / Source:   {origin} / {source}")
    print(f"Rules:             passed={passed}, failed={failed}, total={total}")
    print(f"Files:             scanned={scanned}, skipped={skipped}")
    print(f"Detected formats:  {formats_str}")
    print(f"Labels:            {label_str}")
    print(f"Started:           {fmt_dt(started)}")
    print(f"Created:           {fmt_dt(created)}")
    print(f"Updated:           {fmt_dt(updated)}")
    if hint:
        print("────────────────────────────────────────")
        print(f"Hint:              {hint}")

pretty_scan(scan)

AI Model Security Scan
────────────────────────────────────────
Outcome:           EvalOutcome.ALLOWED
Scan UUID:         07decb6e-f921-4862-a30d-604a651e0874
Security Group:    rteixeira-HF (ed8a56bf-2aed-4af9-a489-5081214f3f67)
Model URI:         https://huggingface.co/nvidia/NVIDIA-Nemotron-3-Nano-30B-A3B-FP8
Origin / Source:   ScanOrigin.HUGGING_FACE / SourceType.HUGGING_FACE
Rules:             passed=11, failed=0, total=11
Files:             scanned=32, skipped=6
Detected formats:  json, not_model, safetensors, safetensors_index, yaml
Labels:            env=production
Started:           2026-02-28T13:35:47.421477+00:00
Created:           2026-02-28T13:35:47.425425+00:00
Updated:           2026-02-28T13:35:47.560890+00:00
